In [10]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [11]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [12]:
#Datasets path

# 3 percent with HYP 10 percent
train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2/Training/'
val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2/Validation/'

# 3 percent with HYP 20 percent
#train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 20 percent - v2/Training/'
#val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 20 percent - v2/Validation/'

# 20 percent
#train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/20 percent/Training/'
#val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/20 percent/Validation/'

# 10 percent
#train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/10 percent/Training/'
#val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/10 percent/Validation/'



#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2--New/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2--New/Validation/"


In [13]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 30246 files belonging to 5 classes.


2023-01-10 15:08:04.230525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 15:08:04.254815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 15:08:04.255154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 15:08:04.270080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 8051 files belonging to 5 classes.


In [14]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [15]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [16]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [17]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [18]:
x= base_model(inputs,training= False)

In [19]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [20]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [21]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [22]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [23]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [24]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [25]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-10 15:08:25.741894: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-10 15:08:26.007010: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


237/237 [==============================] - ETA: 0s - loss: 1.1869 - accuracy: 0.5007 - sensitivity_at_specificity: 0.8966 - specificity_at_sensitivity: 0.8873 - recall: 0.2848 - precision: 0.7053
Epoch 1: val_accuracy improved from -inf to 0.62017, saving model to ECG_Spectrogram_Model.h5
237/237 [==============================] - 200s 795ms/step - loss: 1.1869 - accuracy: 0.5007 - sensitivity_at_specificity: 0.8966 - specificity_at_sensitivity: 0.8873 - recall: 0.2848 - precision: 0.7053 - val_loss: 0.9548 - val_accuracy: 0.6202 - val_sensitivity_at_specificity: 0.9641 - val_specificity_at_sensitivity: 0.9492 - val_recall: 0.4491 - val_precision: 0.7474
Epoch 2/50
237/237 [==============================] - ETA: 0s - loss: 0.8755 - accuracy: 0.6539 - sensitivity_at_specificity: 0.9732 - specificity_at_sensitivity: 0.9617 - recall: 0.5337 - precision: 0.7441
Epoch 2: val_accuracy improved from 0.62017 to 0.65321, saving model to ECG_Spectrogram_Model.h5
237/237 [========================

Epoch 14/50
237/237 [==============================] - ETA: 0s - loss: 0.5572 - accuracy: 0.7885 - sensitivity_at_specificity: 0.9959 - specificity_at_sensitivity: 0.9932 - recall: 0.7404 - precision: 0.8319
Epoch 14: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 728ms/step - loss: 0.5572 - accuracy: 0.7885 - sensitivity_at_specificity: 0.9959 - specificity_at_sensitivity: 0.9932 - recall: 0.7404 - precision: 0.8319 - val_loss: 0.8198 - val_accuracy: 0.6932 - val_sensitivity_at_specificity: 0.9781 - val_specificity_at_sensitivity: 0.9740 - val_recall: 0.6428 - val_precision: 0.7490
Epoch 15/50
237/237 [==============================] - ETA: 0s - loss: 0.5519 - accuracy: 0.7921 - sensitivity_at_specificity: 0.9959 - specificity_at_sensitivity: 0.9932 - recall: 0.7456 - precision: 0.8365
Epoch 15: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 728ms/step - loss: 0.5519 - accuracy: 0.7921 - sensitivity_

Epoch 27/50
237/237 [==============================] - ETA: 0s - loss: 0.4323 - accuracy: 0.8378 - sensitivity_at_specificity: 0.9983 - specificity_at_sensitivity: 0.9973 - recall: 0.8058 - precision: 0.8666
Epoch 27: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 728ms/step - loss: 0.4323 - accuracy: 0.8378 - sensitivity_at_specificity: 0.9983 - specificity_at_sensitivity: 0.9973 - recall: 0.8058 - precision: 0.8666 - val_loss: 0.9995 - val_accuracy: 0.7081 - val_sensitivity_at_specificity: 0.9562 - val_specificity_at_sensitivity: 0.9724 - val_recall: 0.6777 - val_precision: 0.7338
Epoch 28/50
237/237 [==============================] - ETA: 0s - loss: 0.4412 - accuracy: 0.8337 - sensitivity_at_specificity: 0.9981 - specificity_at_sensitivity: 0.9970 - recall: 0.8025 - precision: 0.8630
Epoch 28: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 728ms/step - loss: 0.4412 - accuracy: 0.8337 - sensitivity_

Epoch 40/50
237/237 [==============================] - ETA: 0s - loss: 0.3255 - accuracy: 0.8785 - sensitivity_at_specificity: 0.9990 - specificity_at_sensitivity: 0.9992 - recall: 0.8590 - precision: 0.8978
Epoch 40: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 728ms/step - loss: 0.3255 - accuracy: 0.8785 - sensitivity_at_specificity: 0.9990 - specificity_at_sensitivity: 0.9992 - recall: 0.8590 - precision: 0.8978 - val_loss: 1.1321 - val_accuracy: 0.6989 - val_sensitivity_at_specificity: 0.9440 - val_specificity_at_sensitivity: 0.9719 - val_recall: 0.6794 - val_precision: 0.7164
Epoch 41/50
237/237 [==============================] - ETA: 0s - loss: 0.3287 - accuracy: 0.8755 - sensitivity_at_specificity: 0.9986 - specificity_at_sensitivity: 0.9992 - recall: 0.8555 - precision: 0.8956
Epoch 41: val_accuracy did not improve from 0.72898
237/237 [==============================] - 173s 729ms/step - loss: 0.3287 - accuracy: 0.8755 - sensitivity_

In [26]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8973087072372437
Validation Accuracy:  0.7289777398109436
Validation Specificity:  0.9811514019966125
Validation Sensitivity:  0.9823624491691589
Validation Recall:  0.6969320774078369
Validation Precision:  0.7724354267120361
Validation Loss:  0.7525846362113953


In [27]:
### Calculating feature wise similarties from the trained model

In [42]:
#importing libraries 

from keras.models import Model
#from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import os
import glob
from glob import iglob
from IPython.display import HTML, display
from pathlib import Path
from IPython.display import clear_output
import cv2

In [43]:
def image_processing(img_array):
    """ Preprocess image to be used in a keras model instance
    Args:
        img_array: Numpy array of an image which will be predicte
    Returns:
        processed_img = Numpy array which represents the processed image
    """    
    # Expand the shape
    img = np.expand_dims(img_array, axis=0)
    processed_img = img/255

    return processed_img

In [44]:
def get_feature_vector(model, img_path):
    """ Get a feature vector extraction from an image by using a keras model instance
    Args:
        model: Keras model instance used to do the classification.
        img_path: String to the image path which will be predicted
    Returns:
        feature_vect: List of visual feature from the input image
    """
    
    # Creation of a new keras model instance without the last layer
    layername_feature_extraction = "dense_2"  #extraction of the second last layer of the trained model
    model_feature_vect = Model(inputs=model.input, outputs=model.get_layer(layername_feature_extraction).output)

    # Image processing
    img_size_model = (256,256)
    img = image.load_img(img_path, target_size=img_size_model)
    img_arr = np.array(img)
    img_ = image_processing(img_arr)
    
    # Visual feature extraction
    feature_vect = model_feature_vect.predict(img_)

    return feature_vect

In [45]:
from scipy import spatial

def calculate_similarity(vector1, vector2):
    """Compute similarities between two images using 'cosine similarities'
    Args:
        vector1: Numpy vector to represent feature extracted vector from image 1
        vector2: Numpy vector to represent feature extracted vector from image 2
    Returns:
        sim_cos: Float to describe the similarity between both images
    """
    sim_cos = 1-spatial.distance.cosine(vector1, vector2)
    
    return sim_cos

In [46]:
def create_folder(folder_name):
    """ Create folder if there is not
    Args:
        folder_name: String, folder name
    Returns:
        None
    """
    if not os.path.isdir(f"/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/for_image_similarity/{folder_name}"):
        os.makedirs(f"/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/for_image_similarity/{folder_name}")
       

In [47]:
def compute_similarity_img(model, img_path_1, img_path_2):
    """ Return a cosine similarity between both images and display them in HTML
    Args:
        model: Keras model instance used to do the feature extraction
        img_path_1: String to the image 1 path
        img_path_2: String to the image 2 path
    Returns:
        sim_cos: Float to describe the similarity between both images
    """
    filename1 = os.path.basename(img_path_1).split(".")[0]
    filename2 = os.path.basename(img_path_2).split(".")[0]
    
    # Compute feature vector extracted
    fea_vec_img1 = get_feature_vector(model, img_path_1)
    fea_vec_img2 = get_feature_vector(model, img_path_2)
    
    # Compute cosine similarity
    sim_cos = calculate_similarity(fea_vec_img1, fea_vec_img2)
    
    # Read images
    img_size_model = (256,256) #get_img_size_model(model)
    im1 = cv2.resize(cv2.imread(img_path_1), dsize=img_size_model, interpolation = cv2.INTER_AREA)
    im2 = cv2.resize(cv2.imread(img_path_2), dsize=img_size_model, interpolation = cv2.INTER_AREA)
    
    # Concatenate images horizontally
    im12 = cv2.hconcat([im1, im2])
    
    # Save concatenated image
    dst_dir_cos_sim = "/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/for_image_similarity"
    create_folder(dst_dir_cos_sim)
    dst_dir = f"{dst_dir_cos_sim}/{model.name}"
    create_folder(dst_dir)
    
    new_filename = f"{filename1}_{filename2}"
    cv2.imwrite(f"{dst_dir}/{new_filename}.jpeg", im12)

    # Display images with cosine similarity result with HTML
    thumb = " ({}) {}/{}: cos_sim = {:.4f}".format(model.name, filename1, filename2, sim_cos)
    #thumb += ''.join( ["" %str(s)
     ##                  for s in sorted(glob('{}/{}.jpg'.format(dst_dir, new_filename))) ])
    
    #display(HTML(thumb))
   # print()
    
    return sim_cos

In [41]:
#input all the augmented images (0 indexed) at once alongwith original images
Images_count=0
similarity_value=0
results_statement=""

disease = ["CD","HYP", "MI", "STTC"] 

for x in disease:
    
    Images_count=0
    similarity_value=0

    src_dir_1 = "/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/for_similarity_measures/Original_graphs_for_similarity/"+x+"/"
    src_dir_2 = "/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/DA_graphs/3 percent with HYP 10 percent/"+x+"/"

    for file in glob.iglob(src_dir_1 + '**/*.jpeg', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]       

        image_1=file
        image_2=src_dir_2+new_file_name+"_0"+".jpeg"

        isExist = os.path.exists(image_2)

        if isExist==True:
            Images_count+=1
            sim_cos_11 = compute_similarity_img(model, image_1, image_2) 
            similarity_value=(similarity_value+sim_cos_11)
            
            if Images_count==200:
                 clear_output(wait=True)
            
    results_statement=results_statement,"===> Average similarity of class ",x," is: ",(similarity_value/Images_count)," with ",Images_count," images.\n"
    
print(results_statement)

1/1 [==============================] - 0s 130ms/step


1/1 [==============================] - 0s 126ms/step


1/1 [==============================] - 0s 123ms/step


1/1 [==============================] - 0s 128ms/step


1/1 [==============================] - 0s 151ms/step


...

1/1 [==============================] - 0s 133ms/step


1/1 [==============================] - 0s 351ms/step


1/1 [==============================] - 0s 124ms/step


1/1 [==============================] - 0s 119ms/step


1/1 [==============================] - 0s 322ms/step
(((('', '===> Average similarity of class ', 'CD', ' is: ', 0.945072927617448, ' with ', 2142, ' images.\n'), '===> Average similarity of class ', 'HYP', ' is: ', 0.7776480483416036, ' with ', 181, ' images.\n'), '===> Average similarity of class ', 'MI', ' is: ', 0.9402222161466901, ' with ', 1001, ' images.\n'), '===> Average similarity of class ', 'STTC', ' is: ', 0.9134377540308536, ' with ', 2331, ' images.\n')
